# 2023 CITS4012 Assignment
*Make sure you change the file name with your student id.*

# Readme
*If there is something to be noted for the marker, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

In [ ]:
import sys
sys.path.append('./src/QA')
from importlib import reload
from data_prep import DataPrep
reload(data_prep)
from data_prep import DataPrep

ModuleNotFoundError: ignored

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [1]:
# only need to run these commands if using Colab
# Mount google drive and change directory to the git repo folder
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/CITS4012_2023
import sys
# colab seems to require an explicit path for this to work from mounted drive
sys.path.append('/content/gdrive/MyDrive/CITS4012_2023/src/QA')

Mounted at /content/gdrive
/content/gdrive/MyDrive/CITS4012_2023


In [ ]:
# when not using colab
import sys
sys.path.append('./src/QA')

In [2]:
from data_prep import DataPrep

In [3]:
df = DataPrep.parse_tsv('WikiQA-train.tsv')
question_id_list = df.loc[:, "QuestionID"]
question_id_list = list(set(question_id_list))
question_id_list.sort() # fix sorting or dont sort

question_id = {}
for i in df.index:
    row = df.loc[i]
    id = row["QuestionID"]
    if id not in question_id:
        question_id[id] = {"document": []}
        question_id[id]["question"] = row["Question"]
    
    question_id[id]["document"].append((row["Sentence"], row["Label"]))

In [4]:
toks = DataPrep.tokenize_question_and_doc(question_id["Q1"])
toks


(['how', 'are', 'glacier', 'caves', 'formed', '?'],
 [['A',
   'partly',
   'submerged',
   'glacier',
   'cave',
   'on',
   'Perito',
   'Moreno',
   'Glacier',
   '.'],
  ['The', 'ice', 'facade', 'is', 'approximately', '60', 'm', 'high'],
  ['Ice', 'formations', 'in', 'the', 'Titlis', 'glacier', 'cave'],
  ['A',
   'glacier',
   'cave',
   'is',
   'a',
   'cave',
   'formed',
   'within',
   'the',
   'ice',
   'of',
   'a',
   'glacier',
   '.'],
  ['Glacier',
   'caves',
   'are',
   'often',
   'called',
   'ice',
   'caves',
   ',',
   'but',
   'this',
   'term',
   'is',
   'properly',
   'used',
   'to',
   'describe',
   'bedrock',
   'caves',
   'that',
   'contain',
   'year-round',
   'ice',
   '.']],
 [['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['BOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA

In [5]:
question_id["Q1"]

{'document': [('A partly submerged glacier cave on Perito Moreno Glacier .',
   0),
  ('The ice facade is approximately 60 m high', 0),
  ('Ice formations in the Titlis glacier cave', 0),
  ('A glacier cave is a cave formed within the ice of a glacier .', 1),
  ('Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice.',
   0)],
 'question': 'how are glacier caves formed?'}

questions that NER tagging works in:
- how many series in hockey before the stanley cup
- how much are the harry potter movies worth

questions it doesn't work in:
- how did anne frank die
- what did benedict arnold die
- how old is kurt cobain's daughter


In [6]:
from word_embed import WordEmbed

[===============================================---] 95.0% 121.7/128.1MB downloaded


In [7]:
from feat_extract import FeatExt

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
from embed_doc import EmbedAndConcat

In [9]:
toks_embedded = EmbedAndConcat.doc_concat(toks[1])

In [10]:
doc_ner_tags = FeatExt.ner_tag_doc(question_id["Q1"]["document"])
toks_embedded_with_ner = EmbedAndConcat.doc_concat(toks[1], with_ner=True, doc_ner_tags=doc_ner_tags)

In [11]:
toks_embedded[0][0]

array([-0.27085999,  0.044006  , -0.02026   , -0.17395   ,  0.6444    ,
        0.71213001,  0.35510001,  0.47138   , -0.29637   ,  0.54426998,
       -0.72294003, -0.0047612 ,  0.040611  ,  0.043236  ,  0.29729   ,
        0.10725   ,  0.40156001, -0.53662002,  0.033382  ,  0.067396  ,
        0.64556003, -0.085523  ,  0.14103   ,  0.094539  ,  0.74947   ,
       -0.19400001, -0.68739003, -0.41740999, -0.22807001,  0.12      ,
       -0.48999   ,  0.80944997,  0.045138  , -0.11898   ,  0.20161   ,
        0.39276001, -0.20121001,  0.31354001,  0.75304002,  0.25907001,
       -0.11566   , -0.029319  ,  0.93498999, -0.36067   ,  0.52420002,
        0.23706   ,  0.52714998,  0.22869   , -0.51958001, -0.79348999,
       -0.20367999, -0.50186998,  0.18748   ,  0.94282001, -0.44834   ,
       -3.67919993,  0.044183  , -0.26751   ,  2.19970012,  0.241     ,
       -0.033425  ,  0.69553   , -0.64472002, -0.0072277 ,  0.89574999,
        0.20015   ,  0.46493   ,  0.61932999, -0.1066    ,  0.08

In [12]:
toks_embedded_with_ner[0][0]

array([-0.27085999,  0.044006  , -0.02026   , -0.17395   ,  0.6444    ,
        0.71213001,  0.35510001,  0.47138   , -0.29637   ,  0.54426998,
       -0.72294003, -0.0047612 ,  0.040611  ,  0.043236  ,  0.29729   ,
        0.10725   ,  0.40156001, -0.53662002,  0.033382  ,  0.067396  ,
        0.64556003, -0.085523  ,  0.14103   ,  0.094539  ,  0.74947   ,
       -0.19400001, -0.68739003, -0.41740999, -0.22807001,  0.12      ,
       -0.48999   ,  0.80944997,  0.045138  , -0.11898   ,  0.20161   ,
        0.39276001, -0.20121001,  0.31354001,  0.75304002,  0.25907001,
       -0.11566   , -0.029319  ,  0.93498999, -0.36067   ,  0.52420002,
        0.23706   ,  0.52714998,  0.22869   , -0.51958001, -0.79348999,
       -0.20367999, -0.50186998,  0.18748   ,  0.94282001, -0.44834   ,
       -3.67919993,  0.044183  , -0.26751   ,  2.19970012,  0.241     ,
       -0.033425  ,  0.69553   , -0.64472002, -0.0072277 ,  0.89574999,
        0.20015   ,  0.46493   ,  0.61932999, -0.1066    ,  0.08

In [13]:
# use for questions
q_embedded = EmbedAndConcat.q_concat(toks[0])

In [14]:
q_ner_tags = FeatExt.ner_tag(question_id["Q1"]["question"])
q_embedded_with_ner = EmbedAndConcat.q_concat(toks[0], with_ner=True, question_ner_tags=q_ner_tags)

In [15]:
q_embedded[0]

array([-2.03140005e-01,  5.04670024e-01, -2.52229989e-01,  3.77880007e-01,
       -6.85999990e-01, -1.87439993e-01,  1.86379999e-01, -2.78019994e-01,
       -9.51690018e-01,  1.68990001e-01,  1.19739997e+00, -1.08120000e+00,
       -3.10339987e-01,  4.56840008e-01,  8.80369991e-02, -8.46429989e-02,
       -5.35300016e-01,  3.64959985e-02, -1.38109997e-01,  1.18000004e-02,
        1.33480000e+00, -3.48879993e-02, -1.65289998e-01,  7.50750005e-01,
        1.89170003e-01,  4.18940008e-01,  6.20199978e-01,  5.20299971e-01,
        1.11620001e-01, -2.45849997e-01,  1.90490007e-01,  1.15460002e+00,
        1.73400000e-01,  3.51419985e-01,  6.07020020e-01,  2.05279998e-02,
        5.24110019e-01,  2.44910002e-01, -4.31690007e-01,  1.48379996e-01,
        1.11329997e+00, -5.97199976e-01, -1.18579999e-01, -3.07709992e-01,
       -2.79399991e-01,  4.52170014e-01, -6.71729982e-01,  4.22639996e-01,
        4.59879994e-01, -2.11520001e-01, -4.26420003e-01,  2.52750009e-01,
        5.33089995e-01,  

In [16]:
q_embedded_with_ner[0]

array([-2.03140005e-01,  5.04670024e-01, -2.52229989e-01,  3.77880007e-01,
       -6.85999990e-01, -1.87439993e-01,  1.86379999e-01, -2.78019994e-01,
       -9.51690018e-01,  1.68990001e-01,  1.19739997e+00, -1.08120000e+00,
       -3.10339987e-01,  4.56840008e-01,  8.80369991e-02, -8.46429989e-02,
       -5.35300016e-01,  3.64959985e-02, -1.38109997e-01,  1.18000004e-02,
        1.33480000e+00, -3.48879993e-02, -1.65289998e-01,  7.50750005e-01,
        1.89170003e-01,  4.18940008e-01,  6.20199978e-01,  5.20299971e-01,
        1.11620001e-01, -2.45849997e-01,  1.90490007e-01,  1.15460002e+00,
        1.73400000e-01,  3.51419985e-01,  6.07020020e-01,  2.05279998e-02,
        5.24110019e-01,  2.44910002e-01, -4.31690007e-01,  1.48379996e-01,
        1.11329997e+00, -5.97199976e-01, -1.18579999e-01, -3.07709992e-01,
       -2.79399991e-01,  4.52170014e-01, -6.71729982e-01,  4.22639996e-01,
        4.59879994e-01, -2.11520001e-01, -4.26420003e-01,  2.52750009e-01,
        5.33089995e-01,  

# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



###3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 